In [1]:
from iexfinance import Stock
from iexfinance import get_available_symbols

In [2]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

#loading the .yml document
trainer = Trainer(config.load("/Users/hejianping/Desktop/rasa_nlu.train/config_spacy.yml"))

# loading the training data
training_data = load_data('/Users/hejianping/Desktop/rasa_nlu.train/demo-rasa.json')

# starting training 
interpreter = trainer.train(training_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


In [3]:
# constant assignment
company_dict = {'apple': 'AAPL', 'tesla': 'TSLA', 'microsoft': 'MSFT'}
answer="123"

INIT = 0
CLP = 1
DENY = 2
VOLUME = 3
VALUE = 4

In [4]:
# setting up the state machine
policy = {
    (INIT, 'close'): (CLP, 'Ok. I am loading...'),
    (INIT, 'none'): (INIT, 'Hi body! What can I do for you.'),
    (CLP, 'deny'): (CLP, 'As you wish. Wait please.'),
    (INIT, 'volume'): (VOLUME, 'Alright, I am loading now...'),
    (VOLUME, 'deny'): (VOLUME, 'Ok. I am dealing with it...'),
    (INIT, 'value'): (VALUE, 'Wait please...'),
    (VALUE, 'deny'): (VALUE, 'Ok. It is not hard...'),
}

In [5]:
# sending message to the charting bot
def send_message(message, policy, state, params, intent_dict):
    print("USER : {}".format(message))
    new_state, response, params = respond(message, policy, state, params, intent_dict)
    return new_state, params

# producing reply according to the message
def respond(message, policy, state, params, intent_dict):
    (new_state, response) = policy[(state, interpret(message))]
    print("BOT : {}".format(response))
    params = analyse(message, params, intent_dict)
    
    # reset the state machine after a denial is processed
    empty = {}
    if new_state == DENY:
        if params == empty:
            new_state = INIT
        else:
            new_state = state
    return new_state, response, params

In [6]:
# intent recognition
def interpret(message):
    intent = interpreter.parse(message)['intent']['name']
    words = [str(i) for i in message.split()]
    for i in range(len(words)):
        if words[i] == 'no' or words[i] == 'not':
            intent = 'deny'
            break
        else:
            continue
    if intent == 'close':
        return 'close'
    elif intent == 'time':
        return 'time' 
    elif intent == 'volume':
        return 'volume'
    elif intent == 'value':
        return 'value'
    elif intent == 'deny':
        return 'deny'   
    else:
        return 'none'

In [7]:
# intent extraction
def analyse(message, params, intent_dict):
    entities = interpreter.parse(message)['entities']
    intent = interpreter.parse(message)['intent']['name']
    words = [str(i) for i in message.split()]
    for i in range(len(words)):
        if words[i] == 'no' or words[i] == 'not':
            intent = 'deny'
            break
        else:
            continue

    # enquiry about close price?
    if intent == 'close':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
            print("BOT : The close price of {0} is {1}".format(params[company], stock_search.get_close()))
        intent_dict.append(intent)
        
    # enquiry about trading volume？
    elif intent == 'volume':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
            print("BOT : The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
        intent_dict.append(intent)
        
    # enquiry about market value?
    elif intent == 'value':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
            print("BOT : The value of {0} is {1}".format(params[company], stock_search.get_financials()[0]['totalAssets']))
        intent_dict.append(intent)
        
    # any denial？
    elif intent == 'deny':
        flag = False
        for i in range(len(words)):
            words[i] = words[i].lower()
            if words[i] == 'no' or words[i] == 'not':
                flag = True
            key = 'company_' + str(words[i])
            if key in params and flag:
                params.pop(key)
                
        # reply to the denial
        if intent_dict[len(intent_dict)-1] == 'close':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
                print("BOT : The close price of {0} is {1}".format(params[company], stock_search.get_close()))
        elif intent_dict[len(intent_dict)-1] == 'value':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
                print("BOT : The value of {0} is {1}".format(params[company], stock_search.get_financials()[0]['totalAssets']))
        elif intent_dict[len(intent_dict)-1] == 'volume':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
                print("BOT : The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
            params = {}
    return params


In [10]:
#some experiment
state = INIT
params = {}
intent_dict = []
state, params = send_message('Hello', policy, state, params, intent_dict)
state, params = send_message('I would like to know the value of tesla and Apple.', policy, state, params, intent_dict)
state, params = send_message('I do not want anything about tesla now.', policy, state, params, intent_dict)
#state, params = send_message('Hello', policy, state, params, intent_dict)
#state, params = send_message('And what about the close price of microsoft and apple?', policy, state, params, intent_dict)
#state, params = send_message("No apple now.", policy, state, params, intent_dict)
#state, params = send_message("How about the trading volume of tesla and microsoft", policy, state, params, intent_dict)
#state, params = send_message("do not show me tesla", policy, state, params, intent_dict)

USER : Hello
BOT : Hi body! What can I do for you.
USER : I would like to know the value of tesla and Apple.
BOT : Wait please...
BOT : The value of tesla is 29262713000
BOT : The value of apple is 365725000000
USER : I do not want anything about tesla now.
BOT : Ok. It is not hard...
BOT : The value of apple is 365725000000


In [ ]:
#building database
import sqlite3

conn = sqlite3.connect('stock.db')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS stock(name text, close double)')
for i in range(len(result)):
    c.execute('INSERT INTO stock(name, close) VALUES(?, ?)', (result[i]['company'], result[i]['close']))
c.execute('commit')

query = 'SELECT * FROM stock'
c.execute(query)
print(c.fetchall())

In [ ]:
#logging in WeChart account
from wxpy import *

bot = Bot()

In [ ]:
#searching a typical friend
my_friend = bot.friends().search('牛')
print(my_friend)

In [ ]:
#call the function to get reply
@bot.register(my_friend)
def reply_my_friend(msg):
    state, params = send_message(str(msg), policy, state, params, intent_dict))
    return answer
bot.join()